In [272]:
%pip install transformers datasets seqeval scikit-learn

In [273]:
import xml.etree.ElementTree as ET
import pandas as pd

def parse_semeval_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []

    for sentence in root.iter('sentence'):
        sent_id = sentence.get('id')
        text = sentence.find('text').text
        aspects = sentence.find('aspectTerms')

        aspect_terms = []
        polarities = []
        if aspects:
            for aspect in aspects.findall('aspectTerm'):
                aspect_terms.append(aspect.get('term'))
                polarities.append(aspect.get('polarity'))

        data.append({
            'id': sent_id,
            'sentence': text,
            'aspect_terms': aspect_terms,
            'polarities': polarities
        })

    return pd.DataFrame(data)

In [274]:
restaurant_df = parse_semeval_xml('/content/Restaurants_Train_v2.xml')
laptop_df = parse_semeval_xml('/content/Laptop_Train_v2.xml')
combined_df = pd.concat([restaurant_df, laptop_df], ignore_index=True)


In [275]:
print(combined_df.iloc[56])

id                                     1623
sentence        The wine list is excellent.
aspect_terms                    [wine list]
polarities                       [positive]
Name: 56, dtype: object


# Step 2: Preprocess for Aspect Term Extraction (ATE)

In [276]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

def tokenize_and_label(sentence, aspect_terms):
    tokens = word_tokenize(sentence)
    labels = ['O'] * len(tokens)
    for aspect in aspect_terms:
        aspect_tokens = word_tokenize(aspect)
        for i in range(len(tokens) - len(aspect_tokens) + 1):
            if tokens[i:i+len(aspect_tokens)] == aspect_tokens:
                labels[i] = 'B-ASPECT'
                for j in range(1, len(aspect_tokens)):
                    labels[i + j] = 'I-ASPECT'
                break
    return tokens, labels

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [277]:
ate_data = [
    {'tokens': tokenize_and_label(row['sentence'], row['aspect_terms'])[0],
     'labels': tokenize_and_label(row['sentence'], row['aspect_terms'])[1]}
    for _, row in combined_df.iterrows()
]
ate_df = pd.DataFrame(ate_data)


# Step 3: Preprocess for Aspect Sentiment Classification (ASC)

In [278]:
asc_data = [
    {'sentence': row['sentence'], 'aspect': aspect, 'polarity': polarity}
    for _, row in combined_df.iterrows()
    for aspect, polarity in zip(row['aspect_terms'], row['polarities'])
]
asc_df = pd.DataFrame(asc_data)

# Step 4: Convert Data to Hugging Face Dataset Format

In [279]:
from datasets import Dataset
ate_dataset = Dataset.from_pandas(ate_df)
asc_dataset = Dataset.from_pandas(asc_df)

# Step 5: Tokenize the Data

In [280]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
label2id = {'O': 0, 'B-ASPECT': 1, 'I-ASPECT': 2}
id2label = {v: k for k, v in label2id.items()}

from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
label2id = {'O': 0, 'B-ASPECT': 1, 'I-ASPECT': 2}
id2label = {v: k for k, v in label2id.items()}

def preprocess_ate(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True,
        return_offsets_mapping=True,
    )

    all_labels = []
    for batch_index, word_ids in enumerate(tokenized_inputs.word_ids(batch_index=i) for i in range(len(examples["tokens"]))):
        original_labels = examples["labels"][batch_index]
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)  # ignore special tokens
            elif word_idx != previous_word_idx:
                aligned_labels.append(label2id.get(original_labels[word_idx], 0))  # B- or I-ASPECT
            else:
                aligned_labels.append(-100)  # subword token
            previous_word_idx = word_idx

        # Padding to match max_length
        aligned_labels = aligned_labels[:128]
        aligned_labels += [-100] * (128 - len(aligned_labels))
        all_labels.append(aligned_labels)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs


ate_dataset = ate_dataset.map(preprocess_ate, batched=True)


Map:   0%|          | 0/6086 [00:00<?, ? examples/s]

In [281]:
def preprocess_asc(examples):
    text = [f"{s} [SEP] {a}" for s, a in zip(examples['sentence'], examples['aspect'])]
    labels = [0 if p == "negative" else 1 if p == "neutral" else 2 for p in examples['polarity']]
    tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=128)
    tokenized['labels'] = labels
    return tokenized

asc_dataset = asc_dataset.map(preprocess_asc, batched=True)


Map:   0%|          | 0/6051 [00:00<?, ? examples/s]

In [282]:
print(ate_dataset[24])

{'tokens': ['I', 'will', 'be', 'going', 'back', 'very', 'soon', '.'], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], 'input_ids': [101, 1045, 2097, 2022, 2183, 2067, 2200, 2574, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [283]:
print(asc_dataset[83])

{'sentence': 'I liked the beer selection!', 'aspect': 'beer selection', 'polarity': 'positive', 'input_ids': [101, 1045, 4669, 1996, 5404, 4989, 999, 102, 5404, 4989, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [284]:
# Step 7: Split datasets
ate_dataset = ate_dataset.shuffle(seed=42)
train_ate_dataset = ate_dataset.select(range(0, int(0.9 * len(ate_dataset))))
eval_ate_dataset = ate_dataset.select(range(int(0.9 * len(ate_dataset)), len(ate_dataset)))

asc_dataset = asc_dataset.shuffle(seed=42)
train_asc_dataset = asc_dataset.select(range(0, int(0.9 * len(asc_dataset))))
eval_asc_dataset = asc_dataset.select(range(int(0.9 * len(asc_dataset)), len(asc_dataset)))


In [285]:
# def hide_labels_in_eval(dataset, label_column):
#     """
#     Function to hide labels in the evaluation dataset
#     by removing the label column before inference.
#     """
#     dataset = dataset.remove_columns([label_column])
#     return dataset

# # Apply to your evaluation datasets
# eval_ate_dataset = hide_labels_in_eval(eval_ate_dataset, 'labels')
# eval_asc_dataset = hide_labels_in_eval(eval_asc_dataset, 'polarity')


In [286]:
print(train_ate_dataset[0])
print(eval_ate_dataset[0])
print(train_asc_dataset[0])
print(eval_asc_dataset[0])

{'tokens': ['Upon', 'entering', ',', 'we', 'were', 'greeted', 'by', 'the', 'owners', ',', 'Steven', 'and', 'Frederick', ',', 'who', 'went', 'out', 'of', 'their', 'way', 'to', 'be', 'more', 'than', 'gracious', 'hosts', '.'], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 1, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], 'input_ids': [101, 2588, 5738, 1010, 2057, 2020, 11188, 2011, 1996, 5608, 101

# Step 6: Set Up LoRA for Fine-Tuning

In [287]:
from peft import LoraConfig, get_peft_model
from transformers import BertForTokenClassification, AutoModelForSequenceClassification

ate_model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label2id), id2label=id2label, label2id=label2id)
ate_lora_model = get_peft_model(ate_model, LoraConfig(r=8, lora_alpha=32, lora_dropout=0.1))

asc_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
asc_lora_model = get_peft_model(asc_model, LoraConfig(r=8, lora_alpha=32, lora_dropout=0.1))


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Step 7: Fine-Tune the Models

In [288]:
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    DefaultDataCollator,
)


In [289]:
# ATE Metrics
import numpy as np
ate_trainer = Trainer(
    model=ate_lora_model,
    args=TrainingArguments(output_dir="./results_ate", per_device_train_batch_size=16, per_device_eval_batch_size=16,
                           num_train_epochs=3, eval_strategy="epoch", save_strategy="epoch", logging_dir="./logs"),
    data_collator=DataCollatorForTokenClassification(tokenizer),
    train_dataset=train_ate_dataset,
    eval_dataset=eval_ate_dataset,
    tokenizer=tokenizer,
)


<ipython-input-289-b4d514c4b09c>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  ate_trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [290]:
# Train
ate_trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,No log
2,0.339700,No log
3,0.169800,No log


TrainOutput(global_step=1029, training_loss=0.25191975983748516, metrics={'train_runtime': 264.2985, 'train_samples_per_second': 62.168, 'train_steps_per_second': 3.893, 'total_flos': 1077072953386752.0, 'train_loss': 0.25191975983748516, 'epoch': 3.0})

In [291]:
# ASC Metrics
asc_trainer = Trainer(
    model=asc_lora_model,
    args=TrainingArguments(output_dir="./results_asc", per_device_train_batch_size=16, per_device_eval_batch_size=16,
                           num_train_epochs=3, eval_strategy="epoch", logging_strategy="steps", logging_steps=10,
                           save_strategy="epoch", logging_dir="./logs"),
    data_collator=DefaultDataCollator(),
    train_dataset=train_asc_dataset,
    eval_dataset=eval_asc_dataset,
    tokenizer=tokenizer
    )

<ipython-input-291-a0d966793348>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  asc_trainer = Trainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [292]:
asc_trainer.train()

Epoch,Training Loss,Validation Loss
1,0.860500,No log
2,0.888700,No log
3,0.737700,No log


TrainOutput(global_step=1023, training_loss=0.8371613403219631, metrics={'train_runtime': 256.8051, 'train_samples_per_second': 63.609, 'train_steps_per_second': 3.984, 'total_flos': 1078189173262080.0, 'train_loss': 0.8371613403219631, 'epoch': 3.0})

# Step 8: Evaluate the Models


In [293]:
print(ate_lora_model)

PeftModel(
  (base_model): LoraModel(
    (model): BertForTokenClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, o

In [294]:
print(asc_lora_model)

PeftModel(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768

## Evaluate ATE model

In [295]:
print(eval_ate_dataset[230])

{'tokens': ['Not', 'worth', 'it', 'one', 'bit', '.'], 'labels': [-100, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], 'input_ids': [101, 2025, 4276, 2009, 2028, 2978, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [296]:
import torch
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification
from sklearn.metrics import classification_report

# 1) Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
model = ate_lora_model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
model.eval()

# 2) Preprocessing fn
def tokenize_and_align_labels(ex):
    tok = tokenizer(
        ex["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )
    labels = []
    word_ids = tok.word_ids()
    prev = None
    for wid in word_ids:
        if wid is None:
            labels.append(-100)
        elif wid != prev:
            labels.append(ex["labels"][wid])
        else:
            labels.append(-100)
        prev = wid
    tok["labels"] = labels
    return tok

# 3) Build & tokenize your eval dataset
raw = Dataset.from_list(eval_ate_dataset)
tokenized = raw.map(tokenize_and_align_labels, batched=False)

# 4) Drop non-tensor columns, set format
tokenized = tokenized.remove_columns(["tokens", "offset_mapping"])
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# 5) DataLoader + collator
collator = DataCollatorForTokenClassification(tokenizer)
loader = DataLoader(tokenized, batch_size=16, collate_fn=collator)

# 6) Run evaluation
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in loader:
        batch = {k: v.to(model.device) for k,v in batch.items()}
        logits = model(**batch).logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        labs  = batch["labels"].cpu().numpy()

        for p, l in zip(preds, labs):
            mask = l != -100
            all_preds.extend(p[mask])
            all_labels.extend(l[mask])

# 7) Report
label_names = ["O", "B-ASP", "I-ASP"]
print(classification_report(
    all_labels,
    all_preds,
    labels=[0,1,2],
    target_names=label_names,
    digits=4
))


Map:   0%|          | 0/609 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

           O     0.9197    0.9298    0.9247      7492
       B-ASP     0.1409    0.1790    0.1577       525
       I-ASP     0.0000    0.0000    0.0000       226

    accuracy                         0.8565      8243
   macro avg     0.3536    0.3696    0.3608      8243
weighted avg     0.8449    0.8565    0.8505      8243



## Evaluate ASC model

In [297]:
print(eval_asc_dataset[0])

{'sentence': 'The menu looked good, except for offering the Chilean Sea Bass, but the server does not offer up the specials that were written on the board outside.', 'aspect': 'Chilean Sea Bass', 'polarity': 'negative', 'input_ids': [101, 1996, 12183, 2246, 2204, 1010, 3272, 2005, 5378, 1996, 12091, 2712, 3321, 1010, 2021, 1996, 8241, 2515, 2025, 3749, 2039, 1996, 19247, 2008, 2020, 2517, 2006, 1996, 2604, 2648, 1012, 102, 12091, 2712, 3321, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [298]:
import torch
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, Dataset

# Your LoRA model
model = asc_lora_model  # already LoRA-wrapped AutoModelForSequenceClassification
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Your label map (update as needed)
label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
id2label = label_map
label2id = {v: k for k, v in label_map.items()}


# Step 1: Create Dataset and Dataloader
class ASCInferenceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "input_ids": torch.tensor(item["input_ids"]),
            "token_type_ids": torch.tensor(item["token_type_ids"]),
            "attention_mask": torch.tensor(item["attention_mask"]),
            "labels": torch.tensor(item["labels"])
        }

# Prepare dataset and dataloader
eval_dataset = ASCInferenceDataset(eval_asc_dataset)
eval_loader = DataLoader(eval_dataset, batch_size=16)

# Step 2: Run Evaluation
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        input_ids = batch["input_ids"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        )

        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

# Step 3: Report Evaluation Metrics
print(classification_report(all_labels, all_preds, target_names=list(label_map.values())))


              precision    recall  f1-score   support

    negative       0.65      0.84      0.73       167
     neutral       0.00      0.00      0.00       111
    positive       0.76      0.90      0.83       328

    accuracy                           0.72       606
   macro avg       0.47      0.58      0.52       606
weighted avg       0.59      0.72      0.65       606



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Step 9: Using the Models


In [299]:
import torch
# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the models to the same device
ate_lora_model.to(device)
print('ate model to', device)
asc_lora_model.to(device)
print('asc model to', device)

ate model to cuda
asc model to cuda


In [300]:
import torch
from transformers import AutoConfig, AutoModelForTokenClassification, AutoTokenizer
from peft import PeftModel

# 1) Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2) Load tokenizer & LoRA‐adapted token‐classification model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config = AutoConfig.from_pretrained(
    model_name,
    num_labels=3,
    # match whatever your adapter used:
    id2label={0: "O", 1: "B-ASPECT", 2: "I-ASPECT"},
    label2id={"O": 0, "B-ASPECT": 1, "I-ASPECT": 2},
)
base = AutoModelForTokenClassification.from_config(config)
ate_lora_model.to(device)
ate_lora_model.eval()

# 3) Prediction fn (collapses WordPieces)
def predict_ate(sentence, model, tokenizer, device):
    enc = tokenizer(
        sentence.split(),
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        padding=True,
    ).to(device)

    with torch.no_grad():
        logits = model(**enc).logits
    preds = logits.argmax(-1).squeeze().cpu().tolist()

    toks = tokenizer.convert_ids_to_tokens(enc.input_ids.squeeze().cpu())
    word_ids = enc.word_ids()
    word_preds, word_tokens = [], []
    prev = None
    for i, w in enumerate(word_ids):
        if w is None:
            continue
        if w != prev:
            word_preds.append(preds[i])
            word_tokens.append(toks[i])
            prev = w

    labels = [model.config.id2label[p] for p in word_preds]
    return word_tokens, labels

# 4) General extractor for any B-*/I-* scheme
def extract_aspects(tokens, labels):
    aspects, current = [], []
    for t, lab in zip(tokens, labels):
        prefix = lab.split("-", 1)[0]
        if prefix == "B":
            if current:
                aspects.append(" ".join(current))
            current = [t]
        elif prefix == "I" and current:
            current.append(t)
        else:
            if current:
                aspects.append(" ".join(current))
                current = []
    if current:
        aspects.append(" ".join(current))
    return aspects

# 5) Test
test_sentences = [
    "The pizza was delicious but the service was bad",
    "The vibe was awesome",
    "The burger was too salty"
]

for sent in test_sentences:
    toks, labs = predict_ate(sent, ate_lora_model, tokenizer, device)
    asp = extract_aspects(toks, labs)
    print(f"Sentence: {sent}")
    print("Tokens: ", toks)
    print("Labels: ", labs)
    print("Extracted Aspect Terms:", asp)
    print("-" * 50)


Sentence: The pizza was delicious but the service was bad
Tokens:  ['the', 'pizza', 'was', 'delicious', 'but', 'the', 'service', 'was', 'bad']
Labels:  ['O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O']
Extracted Aspect Terms: ['pizza', 'service']
--------------------------------------------------
Sentence: The vibe was awesome
Tokens:  ['the', 'vibe', 'was', 'awesome']
Labels:  ['O', 'B-ASPECT', 'O', 'O']
Extracted Aspect Terms: ['vibe']
--------------------------------------------------
Sentence: The burger was too salty
Tokens:  ['the', 'burger', 'was', 'too', 'salty']
Labels:  ['O', 'B-ASPECT', 'O', 'O', 'O']
Extracted Aspect Terms: ['burger']
--------------------------------------------------


In [301]:
# Dummy APC model setup (replace with your actual model)
apc_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
apc_model = asc_lora_model
apc_model.eval()

# Sentiment map (adjust according to your model's output)
id2sentiment = {0: 'negative', 1: 'neutral', 2: 'positive'}

# Predict sentiment for a given aspect term within its sentence
def predict_apc(sentence, aspect, model, tokenizer, device):
    # Format input as required by your model (e.g., "[CLS] sentence [SEP] aspect [SEP]")
    inputs = tokenizer(
        sentence,
        aspect,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=128
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        pred = torch.argmax(outputs.logits, dim=-1).item()

    return id2sentiment[pred]


In [302]:
predict_apc('The pizza was delicious but the service was bad',
            'pizza',
            asc_lora_model,
            tokenizer,
            device
            )

'negative'

In [303]:
def analyze_aspect_sentiments(text, ate_model, ate_tokenizer, apc_model, apc_tokenizer, device, label_map):
    # Step 1: ATE Prediction
    toks, labs = predict_ate(text, ate_lora_model, tokenizer, device)
    asp = extract_aspects(toks, labs)
    # Step 2: APC for each aspect
    aspect_sentiments = {}
    for aspect in asp:
        sentiment = predict_apc(text, aspect, asc_lora_model, apc_tokenizer, device)
        aspect_sentiments[aspect] = sentiment

    return aspect_sentiments

In [304]:
text = "The pizza was delicious but the service was bad"
analyze_aspect_sentiments(text, ate_lora_model, tokenizer, asc_lora_model, tokenizer, device, label_map)

{'pizza': 'negative', 'service': 'negative'}